# Preparación del entorno

In [ ]:
import os
import pandas as pd

In [ ]:
# Aquí se debe poner la carpeta de la computadora local
carpeta_local = "~/buckets/b1/"

# Establezco el Working Directory
os.chdir(os.path.expanduser(carpeta_local))

# Cargo el dataset
archivo_dataset = "./datasets/competencia_03 (base).csv.gz"
dataset = pd.read_csv(archivo_dataset)


# Prueba con SQL

In [ ]:
%%bash
pip install --upgrade duckdb
pip install --upgrade jupysql
pip install --upgrade duckdb-engine

In [ ]:
import duckdb
import pandas as pd

%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb:///:default:

# Construcción de Clase_ternaria

In [ ]:
%%sql
create or replace table competencia_03 as
select
    *
from read_csv_auto("./datasets/competencia_03 (base).csv.gz")

In [ ]:
%sql select * from competencia_03 limit 5

In [ ]:
%%sql
select
    foto_mes
    , count(*) as cantidad -- cuenta cuantos casos hay en cada foto_mes
                           -- y lo guarda en un campo llamado cantidad
from competencia_03
group by foto_mes

# Descarga índice de inflación

## Armado del índice

In [ ]:
!pip install pandas requests

In [ ]:
import requests
from io import BytesIO

In [ ]:
# URL del archivo
url = "https://www.indec.gob.ar/ftp/cuadros/economia/sh_ipc_08_23.xls"

# Descargar el archivo y cargar los datos de la hoja correcta
response = requests.get(url)
xls = pd.ExcelFile(BytesIO(response.content))

# Cargar los datos desde la hoja de interés
inflacion = pd.read_excel(xls, sheet_name="Índices IPC Cobertura Nacional", skiprows=5, usecols="B:CC")

In [ ]:
inflacion

In [ ]:
# Transponer los datos y establecer la tercera fila como encabezado
inflacion = inflacion.transpose()


In [ ]:
inflacion = inflacion[3]

In [ ]:
inflacion

In [ ]:
# Pasar a data frame
import pandas as pd

inflacion = pd.DataFrame(inflacion)

In [ ]:
inflacion.index.name='foto_mes'

In [ ]:
inflacion

In [ ]:
# Renombrar la columna
inflacion.columns = ["indice_inflacion"]

# Eliminar las filas con valores nulos
inflacion.dropna(inplace=True)

# Eliminar guiones y los últimos dos dígitos de la columna "foto_mes"
inflacion.index = inflacion.index.astype(str).str.replace('-', '').str[:-2]

# Mostrar las primeras filas del DataFrame resultante
inflacion.head()

## Inflación A Sql

In [ ]:
# Guardar el DataFrame como un archivo CSV
inflacion.to_csv('inflacion.csv')

In [ ]:
%%sql
create or replace table inflacion as
select
    *
from read_csv_auto('inflacion.csv')

In [ ]:
%%sql
select *
from inflacion

Agrego inflación acumulada hasta 202107 para hacer el drifting

In [ ]:
%%sql
ALTER TABLE inflacion
ADD COLUMN indice_inflacion_acumulada FLOAT; -- Adjust the data type as needed

WITH InflacionAcumulada AS (
    SELECT
        foto_mes,
        indice_inflacion,
        (
            (SELECT indice_inflacion FROM inflacion WHERE foto_mes = '202107') / indice_inflacion
        ) AS inflacion_acumulada
    FROM
        inflacion
)
UPDATE inflacion
SET indice_inflacion_acumulada = (SELECT inflacion_acumulada FROM InflacionAcumulada WHERE inflacion.foto_mes = InflacionAcumulada.foto_mes);


In [ ]:
%%sql
select *
from inflacion

# Inner join competencia_03 e inflación

In [ ]:
%%sql
CREATE TABLE competencia_03_temp AS
SELECT
    *
FROM
    competencia_03 c2
INNER JOIN
    inflacion i
ON
    c2.foto_mes = i.foto_mes;

In [ ]:
%%sql
-- Elimina la columna 'indice_inflacion' de la tabla temporal
ALTER TABLE competencia_03_temp
DROP COLUMN indice_inflacion;

-- Borra la tabla original 'competencia_03'
DROP TABLE competencia_03;

-- Renombra la tabla temporal a 'competencia_03'
ALTER TABLE competencia_03_temp
RENAME TO competencia_03;

In [ ]:
%%sql
SELECT
    *
FROM
    competencia_03 c2

# Solución del Data Drifting a variables monetarias

Primero, ajustos todas las variables monetarias por la inflación acumulada a 202107

In [ ]:
%%sql
ALTER TABLE competencia_03
ADD COLUMN Mrentabilidad_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN mrentabilidad_annual_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN mcomisiones_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN mactivos_margen_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN mpasivos_margen_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN mcuenta_corriente_adicional_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN mcuenta_corriente_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN mcaja_ahorro_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN mcaja_ahorro_adicional_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN mcaja_ahorro_dolares_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN mcuentas_saldo_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN mautoservicio_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN mtarjeta_visa_consumo_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN mtarjeta_master_consumo_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN mprestamos_personales_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN mprestamos_prendarios_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN mprestamos_hipotecarios_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN mplazo_fijo_dolares_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN mplazo_fijo_pesos_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN minversion1_pesos_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN minversion1_dolares_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN minversion2_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN mpayroll_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN mpayroll2_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN mcuenta_debitos_automaticos_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN mttarjeta_master_debitos_automaticos_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN mpagodeservicios_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN mpagomiscuentas_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN mcajeros_propios_descuentos_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN mtarjeta_visa_descuentos_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN mtarjeta_master_descuentos_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN mcomisiones_mantenimiento_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN mcomisiones_otras_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN mforex_buy_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN mforex_sell_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN mtransferencias_recibidas_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN mtransferencias_emitidas_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN mextraccion_autoservicio_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN mcheques_depositados_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN mcheques_emitidos_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN mcheques_depositados_rechazados_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN mcheques_emitidos_rechazados_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN matm_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN matm_other_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN Master_mfinanciacion_limite_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN Master_msaldototal_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN Master_msaldopesos_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN Master_msaldodolares_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN Master_mconsumospesos_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN Master_mconsumosdolares_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN Master_mlimitecompra_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN Master_madelantopesos_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN Master_madelantodolares_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN Master_mpagado_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN Master_mpagospesos_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN Master_mpagosdolares_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN Master_mconsumototal_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN Master_mpagominimo_ajustado DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN Visa_mfinanciacion_limite_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN Visa_msaldototal_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN Visa_msaldopesos_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN Visa_msaldodolares_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN Visa_mconsumospesos_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN Visa_mconsumosdolares_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN Visa_mlimitecompra_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN Visa_madelantopesos_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN Visa_madelantodolares_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN Visa_mpagado_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN Visa_mpagospesos_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN Visa_mpagosdolares_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN Visa_mconsumototal_ajustada DECIMAL(18, 2);

ALTER TABLE competencia_03
ADD COLUMN Visa_mpagominimo_ajustado DECIMAL(18, 2);

In [ ]:
%%sql
UPDATE competencia_03
SET
    Mrentabilidad_ajustada = Mrentabilidad * indice_inflacion_acumulada,
    mrentabilidad_annual_ajustada = mrentabilidad_annual * indice_inflacion_acumulada,
    mcomisiones_ajustada = mcomisiones * indice_inflacion_acumulada,
    mactivos_margen_ajustada = mactivos_margen * indice_inflacion_acumulada,
    mpasivos_margen_ajustada = mpasivos_margen * indice_inflacion_acumulada,
    mcuenta_corriente_adicional_ajustada = mcuenta_corriente_adicional * indice_inflacion_acumulada,
    mcuenta_corriente_ajustada = mcuenta_corriente * indice_inflacion_acumulada,
    mcaja_ahorro_ajustada = mcaja_ahorro * indice_inflacion_acumulada,
    mcaja_ahorro_adicional_ajustada = mcaja_ahorro_adicional * indice_inflacion_acumulada,
    mcaja_ahorro_dolares_ajustada = mcaja_ahorro_dolares * indice_inflacion_acumulada,
    mcuentas_saldo_ajustada = mcuentas_saldo * indice_inflacion_acumulada,
    mautoservicio_ajustada = mautoservicio * indice_inflacion_acumulada,
    mtarjeta_visa_consumo_ajustada = mtarjeta_visa_consumo * indice_inflacion_acumulada,
    mtarjeta_master_consumo_ajustada = mtarjeta_master_consumo * indice_inflacion_acumulada,
    mprestamos_personales_ajustada = mprestamos_personales * indice_inflacion_acumulada,
    mprestamos_prendarios_ajustada = mprestamos_prendarios * indice_inflacion_acumulada,
    mprestamos_hipotecarios_ajustada = mprestamos_hipotecarios * indice_inflacion_acumulada,
    mplazo_fijo_dolares_ajustada = mplazo_fijo_dolares * indice_inflacion_acumulada,
    mplazo_fijo_pesos_ajustada = mplazo_fijo_pesos * indice_inflacion_acumulada,
    minversion1_pesos_ajustada = minversion1_pesos * indice_inflacion_acumulada,
    minversion1_dolares_ajustada = minversion1_dolares * indice_inflacion_acumulada,
    minversion2_ajustada = minversion2 * indice_inflacion_acumulada,
    mpayroll_ajustada = mpayroll * indice_inflacion_acumulada,
    mpayroll2_ajustada = mpayroll2 * indice_inflacion_acumulada,
    mcuenta_debitos_automaticos_ajustada = mcuenta_debitos_automaticos * indice_inflacion_acumulada,
    mttarjeta_master_debitos_automaticos_ajustada = mttarjeta_master_debitos_automaticos * indice_inflacion_acumulada,
    mpagodeservicios_ajustada = mpagodeservicios * indice_inflacion_acumulada,
    mpagomiscuentas_ajustada = mpagomiscuentas * indice_inflacion_acumulada,
    mcajeros_propios_descuentos_ajustada = mcajeros_propios_descuentos * indice_inflacion_acumulada,
    mtarjeta_visa_descuentos_ajustada = mtarjeta_visa_descuentos * indice_inflacion_acumulada,
    mtarjeta_master_descuentos_ajustada = mtarjeta_master_descuentos * indice_inflacion_acumulada,
    mcomisiones_mantenimiento_ajustada = mcomisiones_mantenimiento * indice_inflacion_acumulada,
    mcomisiones_otras_ajustada = mcomisiones_otras * indice_inflacion_acumulada,
    mforex_buy_ajustada = mforex_buy * indice_inflacion_acumulada,
    mforex_sell_ajustada = mforex_sell * indice_inflacion_acumulada,
    mtransferencias_recibidas_ajustada = mtransferencias_recibidas * indice_inflacion_acumulada,
    mtransferencias_emitidas_ajustada = mtransferencias_emitidas * indice_inflacion_acumulada,
    mextraccion_autoservicio_ajustada = mextraccion_autoservicio * indice_inflacion_acumulada,
    mcheques_depositados_ajustada = mcheques_depositados * indice_inflacion_acumulada,
    mcheques_emitidos_ajustada = mcheques_emitidos * indice_inflacion_acumulada,
    mcheques_depositados_rechazados_ajustada = mcheques_depositados_rechazados * indice_inflacion_acumulada,
    mcheques_emitidos_rechazados_ajustada = mcheques_emitidos_rechazados * indice_inflacion_acumulada,
    matm_ajustada = matm * indice_inflacion_acumulada,
    matm_other_ajustada = matm_other * indice_inflacion_acumulada,
    Master_mfinanciacion_limite_ajustada = Master_mfinanciacion_limite * indice_inflacion_acumulada,
    Master_msaldototal_ajustada = Master_msaldototal * indice_inflacion_acumulada,
    Master_msaldopesos_ajustada = Master_msaldopesos * indice_inflacion_acumulada,
    Master_msaldodolares_ajustada = Master_msaldodolares * indice_inflacion_acumulada,
    Master_mconsumospesos_ajustada = Master_mconsumospesos * indice_inflacion_acumulada,
    Master_mconsumosdolares_ajustada = Master_mconsumosdolares * indice_inflacion_acumulada,
    Master_mlimitecompra_ajustada = Master_mlimitecompra * indice_inflacion_acumulada,
    Master_madelantopesos_ajustada = Master_madelantopesos * indice_inflacion_acumulada,
    Master_madelantodolares_ajustada = Master_madelantodolares * indice_inflacion_acumulada,
    Master_mpagado_ajustada = Master_mpagado * indice_inflacion_acumulada,
    Master_mpagospesos_ajustada = Master_mpagospesos * indice_inflacion_acumulada,
    Master_mpagosdolares_ajustada = Master_mpagosdolares * indice_inflacion_acumulada,
    Master_mconsumototal_ajustada = Master_mconsumototal * indice_inflacion_acumulada,
    Master_mpagominimo_ajustado = Master_mpagominimo * indice_inflacion_acumulada,
    Visa_mfinanciacion_limite_ajustada = Visa_mfinanciacion_limite * indice_inflacion_acumulada,
    Visa_msaldototal_ajustada = Visa_msaldototal * indice_inflacion_acumulada,
    Visa_msaldopesos_ajustada = Visa_msaldopesos * indice_inflacion_acumulada,
    Visa_msaldodolares_ajustada = Visa_msaldodolares * indice_inflacion_acumulada,
    Visa_mconsumospesos_ajustada = Visa_mconsumospesos * indice_inflacion_acumulada,
    Visa_mconsumosdolares_ajustada = Visa_mconsumosdolares * indice_inflacion_acumulada,
    Visa_mlimitecompra_ajustada = Visa_mlimitecompra * indice_inflacion_acumulada,
    Visa_madelantopesos_ajustada = Visa_madelantopesos * indice_inflacion_acumulada,
    Visa_madelantodolares_ajustada = Visa_madelantodolares * indice_inflacion_acumulada,
    Visa_mpagado_ajustada = Visa_mpagado * indice_inflacion_acumulada,
    Visa_mpagospesos_ajustada = Visa_mpagospesos * indice_inflacion_acumulada,
    Visa_mpagosdolares_ajustada = Visa_mpagosdolares * indice_inflacion_acumulada,
    Visa_mconsumototal_ajustada = Visa_mconsumototal * indice_inflacion_acumulada,
    Visa_mpagominimo_ajustado = Visa_mpagominimo * indice_inflacion_acumulada;


Después,ya elimino la variable "inflación_acumulada" ya que no es de más utilidad, y las variables monetarias nominales

In [ ]:
%%sql
ALTER TABLE competencia_03
DROP COLUMN indice_inflacion_acumulada;

In [ ]:
%%sql
ALTER TABLE competencia_03
DROP COLUMN Mrentabilidad;

ALTER TABLE competencia_03
DROP COLUMN mrentabilidad_annual;

ALTER TABLE competencia_03
DROP COLUMN mcomisiones;

ALTER TABLE competencia_03
DROP COLUMN mactivos_margen;

ALTER TABLE competencia_03
DROP COLUMN mpasivos_margen;

ALTER TABLE competencia_03
DROP COLUMN mcuenta_corriente_adicional;

ALTER TABLE competencia_03
DROP COLUMN mcuenta_corriente;

ALTER TABLE competencia_03
DROP COLUMN mcaja_ahorro;

ALTER TABLE competencia_03
DROP COLUMN mcaja_ahorro_adicional;

ALTER TABLE competencia_03
DROP COLUMN mcaja_ahorro_dolares;

ALTER TABLE competencia_03
DROP COLUMN mcuentas_saldo;

ALTER TABLE competencia_03
DROP COLUMN mautoservicio;

ALTER TABLE competencia_03
DROP COLUMN mtarjeta_visa_consumo;

ALTER TABLE competencia_03
DROP COLUMN mtarjeta_master_consumo;

ALTER TABLE competencia_03
DROP COLUMN mprestamos_personales;

ALTER TABLE competencia_03
DROP COLUMN mprestamos_prendarios;

ALTER TABLE competencia_03
DROP COLUMN mprestamos_hipotecarios;

ALTER TABLE competencia_03
DROP COLUMN mplazo_fijo_dolares;

ALTER TABLE competencia_03
DROP COLUMN mplazo_fijo_pesos;

ALTER TABLE competencia_03
DROP COLUMN minversion1_pesos;

ALTER TABLE competencia_03
DROP COLUMN minversion1_dolares;

ALTER TABLE competencia_03
DROP COLUMN minversion2;

ALTER TABLE competencia_03
DROP COLUMN mpayroll;

ALTER TABLE competencia_03
DROP COLUMN mpayroll2;

ALTER TABLE competencia_03
DROP COLUMN mcuenta_debitos_automaticos;

--ALTER TABLE competencia_03
--DROP COLUMN mtarjeta_visa_debitos_automaticos;

ALTER TABLE competencia_03
DROP COLUMN mttarjeta_master_debitos_automaticos;

ALTER TABLE competencia_03
DROP COLUMN mpagodeservicios;

ALTER TABLE competencia_03
DROP COLUMN mpagomiscuentas;

ALTER TABLE competencia_03
DROP COLUMN mcajeros_propios_descuentos;

ALTER TABLE competencia_03
DROP COLUMN mtarjeta_visa_descuentos;

ALTER TABLE competencia_03
DROP COLUMN mtarjeta_master_descuentos;

ALTER TABLE competencia_03
DROP COLUMN mcomisiones_mantenimiento;

ALTER TABLE competencia_03
DROP COLUMN mcomisiones_otras;

ALTER TABLE competencia_03
DROP COLUMN mforex_buy;

ALTER TABLE competencia_03
DROP COLUMN mforex_sell;

ALTER TABLE competencia_03
DROP COLUMN mtransferencias_recibidas;

ALTER TABLE competencia_03
DROP COLUMN mtransferencias_emitidas;

ALTER TABLE competencia_03
DROP COLUMN mextraccion_autoservicio;

ALTER TABLE competencia_03
DROP COLUMN mcheques_depositados;

ALTER TABLE competencia_03
DROP COLUMN mcheques_emitidos;

ALTER TABLE competencia_03
DROP COLUMN mcheques_depositados_rechazados;

ALTER TABLE competencia_03
DROP COLUMN mcheques_emitidos_rechazados;

ALTER TABLE competencia_03
DROP COLUMN matm;

ALTER TABLE competencia_03
DROP COLUMN Master_mfinanciacion_limite;

ALTER TABLE competencia_03
DROP COLUMN Master_msaldototal;

ALTER TABLE competencia_03
DROP COLUMN Master_msaldopesos;

ALTER TABLE competencia_03
DROP COLUMN Master_msaldodolares;

ALTER TABLE competencia_03
DROP COLUMN Master_mconsumospesos;

ALTER TABLE competencia_03
DROP COLUMN Master_mconsumosdolares;

ALTER TABLE competencia_03
DROP COLUMN Master_mlimitecompra;

ALTER TABLE competencia_03
DROP COLUMN Master_madelantopesos;

ALTER TABLE competencia_03
DROP COLUMN Master_madelantodolares;

ALTER TABLE competencia_03
DROP COLUMN Master_mpagado;

ALTER TABLE competencia_03
DROP COLUMN Master_mpagospesos;

ALTER TABLE competencia_03
DROP COLUMN Master_mpagosdolares;

ALTER TABLE competencia_03
DROP COLUMN Master_mconsumototal;

ALTER TABLE competencia_03
DROP COLUMN Master_mpagominimo;

ALTER TABLE competencia_03
DROP COLUMN Visa_mfinanciacion_limite;

ALTER TABLE competencia_03
DROP COLUMN Visa_msaldototal;

ALTER TABLE competencia_03
DROP COLUMN Visa_msaldopesos;

ALTER TABLE competencia_03
DROP COLUMN Visa_msaldodolares;

ALTER TABLE competencia_03
DROP COLUMN Visa_mconsumospesos;

ALTER TABLE competencia_03
DROP COLUMN Visa_mconsumosdolares;

ALTER TABLE competencia_03
DROP COLUMN Visa_mlimitecompra;

ALTER TABLE competencia_03
DROP COLUMN Visa_madelantopesos;

ALTER TABLE competencia_03
DROP COLUMN Visa_madelantodolares;

ALTER TABLE competencia_03
DROP COLUMN Visa_mpagado;

ALTER TABLE competencia_03
DROP COLUMN Visa_mpagospesos;

ALTER TABLE competencia_03
DROP COLUMN Visa_mpagosdolares;

ALTER TABLE competencia_03
DROP COLUMN Visa_mconsumototal;

ALTER TABLE competencia_03
DROP COLUMN Visa_mpagominimo;


In [ ]:
%%sql
select * from competencia_03 limit 5

# Lags 1,3,6

In [ ]:
campos = [['active_quarter', 'cliente_vip', 'internet', 'cliente_edad', 'cliente_antiguedad', 'mrentabilidad_ajustada', 'Mrentabilidad_annual_ajustada', 'mcomisiones_ajustada', 'mactivos_margen_ajustada', 'mpasivos_margen_ajustada', 'cproductos', 'tcuentas', 'mcuenta_corriente_ajustada', 'mcuenta_corriente_adicional_ajustada', 'mcuenta_corriente', 'ccaja_ahorro', 'mcaja_ahorro_ajustada', 'mcaja_ahorro_adicional_ajustada ', 'mcaja_ahorro_dolares_ajustada', 'cdescubierto_preacordado', 'mcuentas_saldo_ajustada', 'ctarjeta_debito', 'ctarjeta_debito_transacciones', 'mautoservicio_ajustada', 'ctarjeta_visa', 'ctarjeta_visa_transacciones', 'mtarjeta_visa_consumo_ajustada', 'ctarjeta_master', 'ctarjeta_master_transacciones', 'mtarjeta_master_consumo_ajustada', 'cprestamos_personales', 'mprestamos_personales_ajustada', 'cprestamos_prendarios', 'mprestamos_prendarios_ajustada', 'cprestamos_hipotecarios', 'mprestamos_hipotecarios_ajustada', 'cplazo_fijo', 'mplazo_fijo_dolares_ajustada', 'mplazo_fijo_pesos_ajustada', 'cinversion1', 'minversion1_pesos_ajustada', 'minversion1_dolares_ajustada', 'cinversion2', 'minversion2_ajustada', 'cseguro_vida', 'cseguro_auto', 'cseguro_vivienda', 'cseguro_accidentes_personales', 'ccaja_seguridad', 'cpayroll_trx', 'mpayroll_ajustada', 'mpayroll2_ajustada', 'cpayroll2_trx', 'ccuenta_debitos_automaticos', 'mcuenta_debitos_automaticos_ajustada', 'ctarjeta_visa_debitos_automaticos', 'mttarjeta_master_debitos_automaticos_ajustada', 'ctarjeta_master_debitos_automaticos', 'mttarjeta_master_debitos_automaticos', 'cpagodeservicios', 'mpagodeservicios_ajustada', 'cpagomiscuentas', 'mpagomiscuentas_ajustada', 'ccajeros_propios_descuentos', 'mcajeros_propios_descuentos_ajustada', 'ctarjeta_visa_descuentos', 'mtarjeta_visa_descuentos_ajustada', 'ctarjeta_master_descuentos', 'mtarjeta_master_descuentos_ajustada', 'ccomisiones_mantenimiento', 'mcomisiones_mantenimiento_ajustada', 'ccomisiones_otras', 'mcomisiones_otras_ajustada', 'cforex', 'cforex_buy', 'mforex_buy_ajustada', 'cforex_sell', 'mforex_sell_ajustada', 'ctransferencias_recibidas', 'mtransferencias_recibidas_ajustada', 'ctransferencias_emitidas', 'mtransferencias_emitidas_ajustada', 'cextraccion_autoservicio', 'mextraccion_autoservicio_ajustada', 'ccheques_depositados', 'mcheques_depositados_ajustada', 'ccheques_emitidos', 'mcheques_emitidos_ajustada', 'ccheques_depositados_rechazados', 'mcheques_depositados_rechazados_ajustada', 'ccheques_emitidos_rechazados', 'mcheques_emitidos_rechazados_ajustada', 'tcallcenter', 'ccallcenter_transacciones', 'thomebanking', 'chomebanking_transacciones', 'ccajas_transacciones', 'ccajas_consultas', 'ccajas_depositos', 'ccajas_extracciones', 'ccajas_otras', 'catm_trx', 'matm_ajustada', 'catm_trx_other', 'matm_other_ajustada', 'ctrx_quarter', 'Master_delinquency', 'Master_status', 'Master_mfinanciacion_limite_ajustada', 'master_Fvencimiento', 'master_Finiciomora', 'Master_msaldototal_ajustada', 'Master_msaldopesos_ajustada', 'Master_msaldodolares_ajustada', 'Master_mconsumospesos_ajustada', 'Master_mconsumosdolares_ajustada', 'Master_mlimitecompra_ajustada', 'Master_madelantopesos_ajustada', 'Master_madelantodolares_ajustada', 'Master_fultimo_cierre', 'Master_mpagado_ajustada', 'Master_mpagospesos_ajustada', 'Master_mpagosdolares_ajustada', 'Master_fechaalta', 'Master_mconsumototal_ajustada', 'Master_cconsumos', 'Master_cadelantosefectivo', 'Master_mpagominimo_ajustado', 'Visa_delinquency', 'Visa_status', 'Visa_mfinanciacion_limite_ajustada', 'Visa_Fvencimiento', 'Visa_Finiciomora', 'Visa_msaldototal_ajustada', 'Visa_msaldopesos_ajustada', 'Visa_msaldodolares_ajustada', 'Visa_mconsumospesos_ajustada', 'Visa_mconsumosdolares_ajustada', 'Visa_mlimitecompra_ajustada', 'Visa_madelantopesos_ajustada', 'Visa_madelantodolares_ajustada', 'Visa_fultimo_cierre', 'Visa_mpagado_ajustada', 'Visa_mpagospesos_ajustada', 'Visa_mpagosdolares_ajustada', 'Visa_fechaalta', 'Visa_mconsumototal_ajustada', 'Visa_cconsumos', 'Visa_cadelantosefectivo', 'Visa_mpagominimo_ajustado']]

nuevos_features = ""
for campo in campos:
  nuevos_features += f", lag({campo}, 1) over (partition by numero_de_cliente order by foto_mes) as {campo}_lag_1"
  nuevos_features += f", lag({campo}, 3) over (partition by numero_de_cliente order by foto_mes) as {campo}_lag_3"
  nuevos_features += f", lag({campo}, 6) over (partition by numero_de_cliente order by foto_mes) as {campo}_lag_6"

print(nuevos_features)

In [ ]:
nuevos_features = ""
for campo in campos:
  nuevos_features += f"\n ,lag({campo}, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_{campo} "
print(nuevos_features)

Uso esta impresión en la siguiente query

In [ ]:
%%sql
create or replace table competencia_03_lags_infla as
select
  numero_de_cliente
  ,foto_mes,
  active_quarter, cliente_vip, internet, cliente_edad, cliente_antiguedad, mrentabilidad_ajustada, Mrentabilidad_annual_ajustada, mcomisiones_ajustada, mactivos_margen_ajustada, mpasivos_margen_ajustada, cproductos, tcuentas, mcuenta_corriente_ajustada, mcuenta_corriente_adicional_ajustada, mcuenta_corriente, ccaja_ahorro, mcaja_ahorro_ajustada, mcaja_ahorro_adicional_ajustada, mcaja_ahorro_dolares_ajustada, cdescubierto_preacordado, mcuentas_saldo_ajustada, ctarjeta_debito, ctarjeta_debito_transacciones, mautoservicio_ajustada, ctarjeta_visa, ctarjeta_visa_transacciones, mtarjeta_visa_consumo_ajustada, ctarjeta_master, ctarjeta_master_transacciones, mtarjeta_master_consumo_ajustada, cprestamos_personales, mprestamos_personales_ajustada, cprestamos_prendarios, mprestamos_prendarios_ajustada, cprestamos_hipotecarios, mprestamos_hipotecarios_ajustada, cplazo_fijo, mplazo_fijo_dolares_ajustada, mplazo_fijo_pesos_ajustada, cinversion1, minversion1_pesos_ajustada, minversion1_dolares_ajustada, cinversion2, minversion2_ajustada, cseguro_vida, cseguro_auto, cseguro_vivienda, cseguro_accidentes_personales ccaja_seguridad, cpayroll_trx, mpayroll_ajustada, mpayroll2_ajustada, cpayroll2_trx, ccuenta_debitos_automaticos, mcuenta_debitos_automaticos_ajustada, ctarjeta_visa_debitos_automaticos, mttarjeta_master_debitos_automaticos_ajustada, ctarjeta_master_debitos_automaticos, mttarjeta_master_debitos_automaticos, cpagodeservicios, mpagodeservicios_ajustada, cpagomiscuentas, mpagomiscuentas_ajustada, ccajeros_propios_descuentos, mcajeros_propios_descuentos_ajustada, ctarjeta_visa_descuentos, mtarjeta_visa_descuentos_ajustada, ctarjeta_master_descuentos, mtarjeta_master_descuentos_ajustada, ccomisiones_mantenimiento, mcomisiones_mantenimiento_ajustada, ccomisiones_otras, mcomisiones_otras_ajustada, cforex, cforex_buy,mforex_buy_ajustada, cforex_sell, mforex_sell_ajustada, ctransferencias_recibidas, mtransferencias_recibidas_ajustada, ctransferencias_emitidas, mtransferencias_emitidas_ajustada, cextraccion_autoservicio, mextraccion_autoservicio_ajustada, ccheques_depositados, mcheques_depositados_ajustada, ccheques_emitidos, mcheques_emitidos_ajustada, ccheques_depositados_rechazados, mcheques_depositados_rechazados_ajustada, ccheques_emitidos_rechazados, mcheques_emitidos_rechazados_ajustada, tcallcenter, ccallcenter_transacciones, thomebanking, chomebanking_transacciones, ccajas_transacciones, ccajas_consultas, ccajas_depositos, ccajas_extracciones, ccajas_otras, catm_trx, matm_ajustada, catm_trx_other, matm_other_ajustada, ctrx_quarter, Master_delinquency, Master_status, Master_mfinanciacion_limite_ajustada, master_Fvencimiento, master_Finiciomora, Master_msaldototal_ajustada, Master_msaldopesos_ajustada, Master_msaldodolares_ajustada, Master_mconsumospesos_ajustada, Master_mconsumosdolares_ajustada, Master_mlimitecompra_ajustada, Master_madelantopesos_ajustada, Master_madelantodolares_ajustada, Master_fultimo_cierre, Master_mpagado_ajustada, Master_mpagospesos_ajustada, Master_mpagosdolares_ajustada, Master_fechaalta, Master_mconsumototal_ajustada, Master_cconsumos, Master_cadelantosefectivo, Master_mpagominimo_ajustado, Visa_delinquency, Visa_status, Visa_mfinanciacion_limite_ajustada, Visa_Fvencimiento, Visa_Finiciomora, Visa_msaldototal_ajustada, Visa_msaldopesos_ajustada, Visa_msaldodolares_ajustada, Visa_mconsumospesos_ajustada, Visa_mconsumosdolares_ajustada, Visa_mlimitecompra_ajustada, Visa_madelantopesos_ajustada, Visa_madelantodolares_ajustada, Visa_fultimo_cierre, Visa_mpagado_ajustada, Visa_mpagospesos_ajustada, Visa_mpagosdolares_ajustada, Visa_fechaalta, Visa_mconsumototal_ajustada, Visa_cconsumos, Visa_cadelantosefectivo, Visa_mpagominimo_ajustado
  ,lag(active_quarter, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_active_quarter
  ,lag(active_quarter, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_active_quarter
  ,lag(active_quarter, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_active_quarter

  ,lag(cliente_vip, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cliente_vip
  ,lag(cliente_vip, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_cliente_vip
  ,lag(cliente_vip, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_cliente_vip

  ,lag(internet, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_internet
  ,lag(internet, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_internet
  ,lag(internet, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_internet

  ,lag(cliente_edad, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cliente_edad
  ,lag(cliente_edad, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_cliente_edad
  ,lag(cliente_edad, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_cliente_edad

  ,lag(cliente_antiguedad, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cliente_antiguedad
  ,lag(cliente_antiguedad, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_cliente_antiguedad
  ,lag(cliente_antiguedad, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_cliente_antiguedad

  ,lag(Mrentabilidad_ajustada, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mrentabilidad
  ,lag(Mrentabilidad_ajustada, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mrentabilidad
  ,lag(Mrentabilidad_ajustada, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_mrentabilidad

  ,lag(mrentabilidad_annual_ajustada, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mrentabilidad_annual
  ,lag(mrentabilidad_annual_ajustada, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mrentabilidad_annual
  ,lag(mrentabilidad_annual_ajustada, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_mrentabilidad_annual

  ,lag(mcomisiones_ajustada, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcomisiones
  ,lag(mcomisiones_ajustada, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mcomisiones
  ,lag(mcomisiones_ajustada, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_mcomisiones

  ,lag(mactivos_margen_ajustada, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mactivos_margen
  ,lag(mactivos_margen_ajustada, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mactivos_margen
  ,lag(mactivos_margen_ajustada, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_mactivos_margen

  ,lag(mpasivos_margen_ajustada, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mpasivos_margen
  ,lag(mpasivos_margen_ajustada, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mpasivos_margen
  ,lag(mpasivos_margen_ajustada, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_mpasivos_margen

  ,lag(cproductos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cproductos
  ,lag(cproductos, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_cproductos
  ,lag(cproductos, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_cproductos

  ,lag(tcuentas, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_tcuentas
  ,lag(tcuentas, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_tcuentas
  ,lag(tcuentas, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_tcuentas

  ,lag(ccuenta_corriente, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ccuenta_corriente
  ,lag(ccuenta_corriente, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_ccuenta_corriente
  ,lag(ccuenta_corriente, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_ccuenta_corriente

  ,lag(mcuenta_corriente_adicional_ajustada, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcuenta_corriente_adicional
  ,lag(mcuenta_corriente_adicional_ajustada, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mcuenta_corriente_adicional
  ,lag(mcuenta_corriente_adicional_ajustada, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_mcuenta_corriente_adicional

  ,lag(mcuenta_corriente_ajustada, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcuenta_corriente
  ,lag(mcuenta_corriente_ajustada, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mcuenta_corriente
  ,lag(mcuenta_corriente_ajustada, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_mcuenta_corriente

  ,lag(ccaja_ahorro, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ccaja_ahorro
  ,lag(ccaja_ahorro, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_ccaja_ahorro
  ,lag(ccaja_ahorro, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_ccaja_ahorro

  ,lag(mcaja_ahorro_ajustada, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcaja_ahorro
  ,lag(mcaja_ahorro_ajustada, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mcaja_ahorro
  ,lag(mcaja_ahorro_ajustada, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_mcaja_ahorro

  ,lag(mcaja_ahorro_adicional_ajustada, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcaja_ahorro_adicional
  ,lag(mcaja_ahorro_adicional_ajustada, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mcaja_ahorro_adicional
  ,lag(mcaja_ahorro_adicional_ajustada, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_mcaja_ahorro_adicional

  ,lag(mcaja_ahorro_dolares_ajustada, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcaja_ahorro_dolares
  ,lag(mcaja_ahorro_dolares_ajustada, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mcaja_ahorro_dolares
  ,lag(mcaja_ahorro_dolares_ajustada, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_mcaja_ahorro_dolares

  ,lag(cdescubierto_preacordado, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cdescubierto_preacordado
  ,lag(cdescubierto_preacordado, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_cdescubierto_preacordado
  ,lag(cdescubierto_preacordado, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_cdescubierto_preacordado

  ,lag(mcuentas_saldo_ajustada, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcuentas_saldo
  ,lag(mcuentas_saldo_ajustada, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mcuentas_saldo
  ,lag(mcuentas_saldo_ajustada, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_mcuentas_saldo

  ,lag(ctarjeta_debito, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ctarjeta_debito
  ,lag(ctarjeta_debito, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_ctarjeta_debito
  ,lag(ctarjeta_debito, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_ctarjeta_debito

  ,lag(ctarjeta_debito_transacciones, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ctarjeta_debito_transacciones
  ,lag(ctarjeta_debito_transacciones, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_ctarjeta_debito_transacciones
  ,lag(ctarjeta_debito_transacciones, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_ctarjeta_debito_transacciones

  ,lag(mautoservicio_ajustada, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mautoservicio
  ,lag(mautoservicio_ajustada, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mautoservicio
  ,lag(mautoservicio_ajustada, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_mautoservicio

  ,lag(ctarjeta_visa, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ctarjeta_visa
  ,lag(ctarjeta_visa, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_ctarjeta_visa
  ,lag(ctarjeta_visa, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_ctarjeta_visa

  ,lag(ctarjeta_visa_transacciones, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ctarjeta_visa_transacciones
  ,lag(ctarjeta_visa_transacciones, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_ctarjeta_visa_transacciones
  ,lag(ctarjeta_visa_transacciones, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_ctarjeta_visa_transacciones

  ,lag(mtarjeta_visa_consumo_ajustada, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mtarjeta_visa_consumo
  ,lag(mtarjeta_visa_consumo_ajustada, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mtarjeta_visa_consumo
  ,lag(mtarjeta_visa_consumo_ajustada, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_mtarjeta_visa_consumo

  ,lag(ctarjeta_master, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ctarjeta_master
  ,lag(ctarjeta_master, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_ctarjeta_master
  ,lag(ctarjeta_master, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_ctarjeta_master

  ,lag(ctarjeta_master_transacciones, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ctarjeta_master_transacciones
  ,lag(ctarjeta_master_transacciones, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_ctarjeta_master_transacciones
  ,lag(ctarjeta_master_transacciones, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_ctarjeta_master_transacciones

  ,lag(mtarjeta_master_consumo_ajustada, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mtarjeta_master_consumo
  ,lag(mtarjeta_master_consumo_ajustada, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mtarjeta_master_consumo
  ,lag(mtarjeta_master_consumo_ajustada, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_mtarjeta_master_consumo

  ,lag(cprestamos_personales, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cprestamos_personales
  ,lag(cprestamos_personales, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_cprestamos_personales
  ,lag(cprestamos_personales, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_cprestamos_personales

  ,lag(mprestamos_personales_ajustada, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mprestamos_personales
  ,lag(mprestamos_personales_ajustada, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mprestamos_personales
  ,lag(mprestamos_personales_ajustada, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_mprestamos_personales

  ,lag(cprestamos_prendarios, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cprestamos_prendarios
  ,lag(cprestamos_prendarios, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_cprestamos_prendarios
  ,lag(cprestamos_prendarios, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_cprestamos_prendarios

  ,lag(mprestamos_prendarios_ajustada, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mprestamos_prendarios
  ,lag(mprestamos_prendarios_ajustada, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mprestamos_prendarios
  ,lag(mprestamos_prendarios_ajustada, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_mprestamos_prendarios

  ,lag(cprestamos_hipotecarios, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cprestamos_hipotecarios
  ,lag(cprestamos_hipotecarios, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_cprestamos_hipotecarios
  ,lag(cprestamos_hipotecarios, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_cprestamos_hipotecarios

  ,lag(mprestamos_hipotecarios_ajustada, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mprestamos_hipotecarios
  ,lag(mprestamos_hipotecarios_ajustada, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mprestamos_hipotecarios
  ,lag(mprestamos_hipotecarios_ajustada, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_mprestamos_hipotecarios

  ,lag(cplazo_fijo, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cplazo_fijo
  ,lag(cplazo_fijo, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_cplazo_fijo
  ,lag(cplazo_fijo, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_cplazo_fijo

  ,lag(mplazo_fijo_dolares_ajustada, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mplazo_fijo_dolares
  ,lag(mplazo_fijo_dolares_ajustada, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mplazo_fijo_dolares
  ,lag(mplazo_fijo_dolares_ajustada, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_mplazo_fijo_dolares

  ,lag(mplazo_fijo_pesos_ajustada, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mplazo_fijo_pesos
  ,lag(mplazo_fijo_pesos_ajustada, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mplazo_fijo_pesos
  ,lag(mplazo_fijo_pesos_ajustada, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_mplazo_fijo_pesos

  ,lag(cinversion1, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cinversion1
  ,lag(cinversion1, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_cinversion1
  ,lag(cinversion1, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_cinversion1

  ,lag(minversion1_pesos_ajustada, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_minversion1_pesos
  ,lag(minversion1_pesos_ajustada, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_minversion1_pesos
  ,lag(minversion1_pesos_ajustada, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_minversion1_pesos

  ,lag(minversion1_dolares_ajustada, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_minversion1_dolares
  ,lag(minversion1_dolares_ajustada, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_minversion1_dolares
  ,lag(minversion1_dolares_ajustada, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_minversion1_dolares

  ,lag(cinversion2, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cinversion2
  ,lag(cinversion2, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_cinversion2
  ,lag(cinversion2, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_cinversion2

  ,lag(minversion2_ajustada, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_minversion2
  ,lag(minversion2_ajustada, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_minversion2
  ,lag(minversion2_ajustada, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_minversion2

  ,lag(cseguro_vida, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cseguro_vida
  ,lag(cseguro_vida, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_cseguro_vida
  ,lag(cseguro_vida, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_cseguro_vida

  ,lag(cseguro_auto, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cseguro_auto
  ,lag(cseguro_auto, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_cseguro_auto
  ,lag(cseguro_auto, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_cseguro_auto

  ,lag(cseguro_vivienda, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cseguro_vivienda
  ,lag(cseguro_vivienda, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_cseguro_vivienda
  ,lag(cseguro_vivienda, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_cseguro_vivienda

  ,lag(cseguro_accidentes_personales, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cseguro_accidentes_personales
  ,lag(cseguro_accidentes_personales, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_cseguro_accidentes_personales
  ,lag(cseguro_accidentes_personales, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_cseguro_accidentes_personales

  ,lag(ccaja_seguridad, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ccaja_seguridad
  ,lag(ccaja_seguridad, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_ccaja_seguridad
  ,lag(ccaja_seguridad, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_ccaja_seguridad

  ,lag(cpayroll_trx, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cpayroll_trx
  ,lag(cpayroll_trx, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_cpayroll_trx
  ,lag(cpayroll_trx, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_cpayroll_trx

  ,lag(mpayroll_ajustada, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mpayroll
  ,lag(mpayroll_ajustada, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mpayroll
  ,lag(mpayroll_ajustada, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_mpayroll

  ,lag(mpayroll2_ajustada, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mpayroll2
  ,lag(mpayroll2_ajustada, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mpayroll2
  ,lag(mpayroll2_ajustada, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_mpayroll2

  ,lag(cpayroll2_trx, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cpayroll2_trx
  ,lag(cpayroll2_trx, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_cpayroll2_trx
  ,lag(cpayroll2_trx, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_cpayroll2_trx

  ,lag(ccuenta_debitos_automaticos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ccuenta_debitos_automaticos
  ,lag(ccuenta_debitos_automaticos, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_ccuenta_debitos_automaticos
  ,lag(ccuenta_debitos_automaticos, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_ccuenta_debitos_automaticos

  ,lag(mcuenta_debitos_automaticos_ajustada, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcuenta_debitos_automaticos
  ,lag(mcuenta_debitos_automaticos_ajustada, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mcuenta_debitos_automaticos
  ,lag(mcuenta_debitos_automaticos_ajustada, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_mcuenta_debitos_automaticos

  ,lag(ctarjeta_visa_debitos_automaticos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ctarjeta_visa_debitos_automaticos
  ,lag(ctarjeta_visa_debitos_automaticos, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_ctarjeta_visa_debitos_automaticos
  ,lag(ctarjeta_visa_debitos_automaticos, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_ctarjeta_visa_debitos_automaticos

  ,lag(mttarjeta_master_debitos_automaticos_ajustada, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mttarjeta_visa_debitos_automaticos
  ,lag(mttarjeta_master_debitos_automaticos_ajustada, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mttarjeta_visa_debitos_automaticos
  ,lag(mttarjeta_master_debitos_automaticos_ajustada, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_mttarjeta_visa_debitos_automaticos

  ,lag(ctarjeta_master_debitos_automaticos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ctarjeta_master_debitos_automaticos
  ,lag(ctarjeta_master_debitos_automaticos, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_ctarjeta_master_debitos_automaticos
  ,lag(ctarjeta_master_debitos_automaticos, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_ctarjeta_master_debitos_automaticos

  ,lag(mttarjeta_master_debitos_automaticos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mttarjeta_master_debitos_automaticos
  ,lag(mttarjeta_master_debitos_automaticos, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mttarjeta_master_debitos_automaticos
  ,lag(mttarjeta_master_debitos_automaticos, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_mttarjeta_master_debitos_automaticos

  ,lag(cpagodeservicios, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cpagodeservicios
  ,lag(cpagodeservicios, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_cpagodeservicios
  ,lag(cpagodeservicios, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_cpagodeservicios

  ,lag(mpagodeservicios_ajustada, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mpagodeservicios
  ,lag(mpagodeservicios_ajustada, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mpagodeservicios
  ,lag(mpagodeservicios_ajustada, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_mpagodeservicios

  ,lag(cpagomiscuentas, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cpagomiscuentas
  ,lag(cpagomiscuentas, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_cpagomiscuentas
  ,lag(cpagomiscuentas, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_cpagomiscuentas

  ,lag(mpagomiscuentas_ajustada, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mpagomiscuentas
  ,lag(mpagomiscuentas_ajustada, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mpagomiscuentas
  ,lag(mpagomiscuentas_ajustada, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_mpagomiscuentas

  ,lag(ccajeros_propios_descuentos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ccajeros_propios_descuentos
  ,lag(ccajeros_propios_descuentos, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_ccajeros_propios_descuentos
  ,lag(ccajeros_propios_descuentos, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_ccajeros_propios_descuentos

  ,lag(mcajeros_propios_descuentos_ajustada, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcajeros_propios_descuentos
  ,lag(mcajeros_propios_descuentos_ajustada, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mcajeros_propios_descuentos
  ,lag(mcajeros_propios_descuentos_ajustada, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_mcajeros_propios_descuentos

  ,lag(ctarjeta_visa_descuentos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ctarjeta_visa_descuentos
  ,lag(ctarjeta_visa_descuentos, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_ctarjeta_visa_descuentos
  ,lag(ctarjeta_visa_descuentos, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_ctarjeta_visa_descuentos

  ,lag(mtarjeta_visa_descuentos_ajustada, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mtarjeta_visa_descuentos
  ,lag(mtarjeta_visa_descuentos_ajustada, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mtarjeta_visa_descuentos
  ,lag(mtarjeta_visa_descuentos_ajustada, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_mtarjeta_visa_descuentos

  ,lag(ctarjeta_master_descuentos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ctarjeta_master_descuentos
  ,lag(ctarjeta_master_descuentos, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_ctarjeta_master_descuentos
  ,lag(ctarjeta_master_descuentos, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_ctarjeta_master_descuentos

  ,lag(mtarjeta_master_descuentos_ajustada, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mtarjeta_master_descuentos
  ,lag(mtarjeta_master_descuentos_ajustada, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mtarjeta_master_descuentos
  ,lag(mtarjeta_master_descuentos_ajustada, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_mtarjeta_master_descuentos

  ,lag(ccomisiones_mantenimiento, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ccomisiones_mantenimiento
  ,lag(ccomisiones_mantenimiento, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_ccomisiones_mantenimiento
  ,lag(ccomisiones_mantenimiento, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_ccomisiones_mantenimiento

  ,lag(mcomisiones_mantenimiento_ajustada, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcomisiones_mantenimiento
  ,lag(mcomisiones_mantenimiento_ajustada, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mcomisiones_mantenimiento
  ,lag(mcomisiones_mantenimiento_ajustada, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_mcomisiones_mantenimiento

  ,lag(ccomisiones_otras, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ccomisiones_otras
  ,lag(ccomisiones_otras, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_ccomisiones_otras
  ,lag(ccomisiones_otras, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_ccomisiones_otras

  ,lag(mcomisiones_otras_ajustada, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcomisiones_otras
  ,lag(mcomisiones_otras_ajustada, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mcomisiones_otras
  ,lag(mcomisiones_otras_ajustada, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_mcomisiones_otras

  ,lag(cforex, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cforex
  ,lag(cforex, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_cforex
  ,lag(cforex, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_cforex

  ,lag(cforex_buy, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cforex_buy
  ,lag(cforex_buy, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_cforex_buy
  ,lag(cforex_buy, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_cforex_buy

  ,lag(mforex_buy_ajustada, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mforex_buy
  ,lag(mforex_buy_ajustada, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mforex_buy
  ,lag(mforex_buy_ajustada, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_mforex_buy

  ,lag(cforex_sell, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cforex_sell
  ,lag(cforex_sell, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_cforex_sell
  ,lag(cforex_sell, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_cforex_sell

  ,lag(mforex_sell_ajustada, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mforex_sell
  ,lag(mforex_sell_ajustada, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mforex_sell
  ,lag(mforex_sell_ajustada, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_mforex_sell

  ,lag(ctransferencias_recibidas, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ctransferencias_recibidas
  ,lag(ctransferencias_recibidas, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_ctransferencias_recibidas
  ,lag(ctransferencias_recibidas, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_ctransferencias_recibidas

  ,lag(mtransferencias_recibidas_ajustada, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mtransferencias_recibidas
  ,lag(mtransferencias_recibidas_ajustada, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mtransferencias_recibidas
  ,lag(mtransferencias_recibidas_ajustada, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_mtransferencias_recibidas

  ,lag(ctransferencias_emitidas, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ctransferencias_emitidas
  ,lag(ctransferencias_emitidas, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_ctransferencias_emitidas
  ,lag(ctransferencias_emitidas, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_ctransferencias_emitidas

  ,lag(mtransferencias_emitidas_ajustada, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mtransferencias_emitidas
  ,lag(mtransferencias_emitidas_ajustada, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mtransferencias_emitidas
  ,lag(mtransferencias_emitidas_ajustada, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_mtransferencias_emitidas

  ,lag(cextraccion_autoservicio, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cextraccion_autoservicio
  ,lag(cextraccion_autoservicio, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_cextraccion_autoservicio
  ,lag(cextraccion_autoservicio, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_cextraccion_autoservicio

  ,lag(mextraccion_autoservicio_ajustada , 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mextraccion_autoservicio
  ,lag(mextraccion_autoservicio_ajustada , 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mextraccion_autoservicio
  ,lag(mextraccion_autoservicio_ajustada , 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_mextraccion_autoservicio

  ,lag(ccheques_depositados, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ccheques_depositados
  ,lag(ccheques_depositados, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_ccheques_depositados
  ,lag(ccheques_depositados, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_ccheques_depositados

  ,lag(mcheques_depositados_ajustada, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcheques_depositados
  ,lag(mcheques_depositados_ajustada, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mcheques_depositados
  ,lag(mcheques_depositados_ajustada, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_mcheques_depositados

  ,lag(ccheques_emitidos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ccheques_emitidos
  ,lag(ccheques_emitidos, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_ccheques_emitidos
  ,lag(ccheques_emitidos, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_ccheques_emitidos

  ,lag(mcheques_emitidos_ajustada, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcheques_emitidos
  ,lag(mcheques_emitidos_ajustada, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mcheques_emitidos
  ,lag(mcheques_emitidos_ajustada, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_mcheques_emitidos

  ,lag(ccheques_depositados_rechazados, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ccheques_depositados_rechazados
  ,lag(ccheques_depositados_rechazados, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_ccheques_depositados_rechazados
  ,lag(ccheques_depositados_rechazados, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_ccheques_depositados_rechazados

  ,lag(mcheques_depositados_rechazados_ajustada , 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcheques_depositados_rechazados
  ,lag(mcheques_depositados_rechazados_ajustada , 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mcheques_depositados_rechazados
  ,lag(mcheques_depositados_rechazados_ajustada , 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_mcheques_depositados_rechazados

  ,lag(ccheques_emitidos_rechazados, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ccheques_emitidos_rechazados
  ,lag(ccheques_emitidos_rechazados, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_ccheques_emitidos_rechazados
  ,lag(ccheques_emitidos_rechazados, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_ccheques_emitidos_rechazados

  ,lag(mcheques_emitidos_rechazados_ajustada , 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcheques_emitidos_rechazados
  ,lag(mcheques_emitidos_rechazados_ajustada , 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mcheques_emitidos_rechazados
  ,lag(mcheques_emitidos_rechazados_ajustada , 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_mcheques_emitidos_rechazados

  ,lag(tcallcenter, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_tcallcenter
  ,lag(tcallcenter, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_tcallcenter
  ,lag(tcallcenter, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_tcallcenter

  ,lag(ccallcenter_transacciones, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ccallcenter_transacciones
  ,lag(ccallcenter_transacciones, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_ccallcenter_transacciones
  ,lag(ccallcenter_transacciones, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_ccallcenter_transacciones

  ,lag(thomebanking, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_thomebanking
  ,lag(thomebanking, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_thomebanking
  ,lag(thomebanking, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_thomebanking

  ,lag(chomebanking_transacciones, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_chomebanking_transacciones
  ,lag(chomebanking_transacciones, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_chomebanking_transacciones
  ,lag(chomebanking_transacciones, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_chomebanking_transacciones

  ,lag(ccajas_transacciones, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ccajas_transacciones
  ,lag(ccajas_transacciones, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_ccajas_transacciones
  ,lag(ccajas_transacciones, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_ccajas_transacciones

  ,lag(ccajas_consultas, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ccajas_consultas
  ,lag(ccajas_consultas, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_ccajas_consultas
  ,lag(ccajas_consultas, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_ccajas_consultas

  ,lag(ccajas_depositos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ccajas_depositos
  ,lag(ccajas_depositos, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_ccajas_depositos
  ,lag(ccajas_depositos, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_ccajas_depositos

  ,lag(ccajas_extracciones, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ccajas_extracciones
  ,lag(ccajas_extracciones, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_ccajas_extracciones
  ,lag(ccajas_extracciones, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_ccajas_extracciones

  ,lag(ccajas_otras, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ccajas_otras
  ,lag(ccajas_otras, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_ccajas_otras
  ,lag(ccajas_otras, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_ccajas_otras

  ,lag(catm_trx, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_catm_trx
  ,lag(catm_trx, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_catm_trx
  ,lag(catm_trx, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_catm_trx

  ,lag(matm_ajustada, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_matm
  ,lag(matm_ajustada, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_matm
  ,lag(matm_ajustada, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_matm

  ,lag(catm_trx_other, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_catm_trx_other
  ,lag(catm_trx_other, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_catm_trx_other
  ,lag(catm_trx_other, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_catm_trx_other

  ,lag(matm_other_ajustada, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_matm_other
  ,lag(matm_other_ajustada, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_matm_other
  ,lag(matm_other_ajustada, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_matm_other

  ,lag(ctrx_quarter, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ctrx_quarter
  ,lag(ctrx_quarter, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_ctrx_quarter
  ,lag(ctrx_quarter, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_ctrx_quarter

  ,lag(Master_delinquency, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_delinquency
  ,lag(Master_delinquency, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Master_delinquency
  ,lag(Master_delinquency, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_Master_delinquency

  ,lag(Master_status, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_status
  ,lag(Master_status, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Master_status
  ,lag(Master_status, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_Master_status

  ,lag(Master_mfinanciacion_limite_ajustada, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_mfinanciacion_limite
  ,lag(Master_mfinanciacion_limite_ajustada, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Master_mfinanciacion_limite
  ,lag(Master_mfinanciacion_limite_ajustada, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_Master_mfinanciacion_limite

  ,lag(Master_Fvencimiento, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_Fvencimiento
  ,lag(Master_Fvencimiento, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Master_Fvencimiento
  ,lag(Master_Fvencimiento, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_Master_Fvencimiento

  ,lag(Master_Finiciomora, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_Finiciomora
  ,lag(Master_Finiciomora, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Master_Finiciomora
  ,lag(Master_Finiciomora, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_Master_Finiciomora

  ,lag(Master_msaldototal, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_msaldototal
  ,lag(Master_msaldototal, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Master_msaldototal
  ,lag(Master_msaldototal, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_Master_msaldototal

  ,lag(Master_msaldototal_ajustada, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_msaldopesos
  ,lag(Master_msaldototal_ajustada, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Master_msaldopesos
  ,lag(Master_msaldototal_ajustada, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_Master_msaldopesos

  ,lag(Master_msaldodolares_ajustada, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_msaldodolares
  ,lag(Master_msaldodolares_ajustada, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Master_msaldodolares
  ,lag(Master_msaldodolares_ajustada, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_Master_msaldodolares

  ,lag(Master_mconsumospesos_ajustada, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_mconsumospesos
  ,lag(Master_mconsumospesos_ajustada, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Master_mconsumospesos
  ,lag(Master_mconsumospesos_ajustada, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_Master_mconsumospesos

  ,lag(Master_mconsumosdolares_ajustada, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_mconsumosdolares
  ,lag(Master_mconsumosdolares_ajustada, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Master_mconsumosdolares
  ,lag(Master_mconsumosdolares_ajustada, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_Master_mconsumosdolares

  ,lag(Master_mlimitecompra, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_mlimitecompra
  ,lag(Master_mlimitecompra, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Master_mlimitecompra
  ,lag(Master_mlimitecompra, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_Master_mlimitecompra

  ,lag(Master_madelantopesos_ajustada, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_madelantopesos
  ,lag(Master_madelantopesos_ajustada, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Master_madelantopesos
  ,lag(Master_madelantopesos_ajustada, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_Master_madelantopesos

  ,lag(Master_madelantodolares_ajustada, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_madelantodolares
  ,lag(Master_madelantodolares_ajustada, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Master_madelantodolares
  ,lag(Master_madelantodolares_ajustada, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_Master_madelantodolares

  ,lag(Master_fultimo_cierre, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_fultimo_cierre
  ,lag(Master_fultimo_cierre, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Master_fultimo_cierre
  ,lag(Master_fultimo_cierre, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_Master_fultimo_cierre

  ,lag(Master_mpagado_ajustada, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_mpagado
  ,lag(Master_mpagado_ajustada, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Master_mpagado
  ,lag(Master_mpagado_ajustada, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_Master_mpagado

  ,lag(Master_mpagospesos_ajustada, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_mpagospesos
  ,lag(Master_mpagospesos_ajustada, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Master_mpagospesos
  ,lag(Master_mpagospesos_ajustada, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_Master_mpagospesos

  ,lag(Master_mpagosdolares_ajustada, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_mpagosdolares
  ,lag(Master_mpagosdolares_ajustada, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Master_mpagosdolares
  ,lag(Master_mpagosdolares_ajustada, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_Master_mpagosdolares

  ,lag(Master_fechaalta, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_fechaalta
  ,lag(Master_fechaalta, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Master_fechaalta
  ,lag(Master_fechaalta, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_Master_fechaalta

  ,lag(Master_mconsumototal_ajustada, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_mconsumototal
  ,lag(Master_mconsumototal_ajustada, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Master_mconsumototal
  ,lag(Master_mconsumototal_ajustada, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_Master_mconsumototal

  ,lag(Master_cconsumos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_cconsumos
  ,lag(Master_cconsumos, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Master_cconsumos
  ,lag(Master_cconsumos, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_Master_cconsumos

  ,lag(Master_cadelantosefectivo, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_cadelantosefectivo
  ,lag(Master_cadelantosefectivo, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Master_cadelantosefectivo
  ,lag(Master_cadelantosefectivo, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_Master_cadelantosefectivo

  ,lag(Master_mpagominimo_ajustado, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_mpagominimo
  ,lag(Master_mpagominimo_ajustado, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Master_mpagominimo
  ,lag(Master_mpagominimo_ajustado, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_Master_mpagominimo

  ,lag(Visa_delinquency, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_delinquency
  ,lag(Visa_delinquency, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Visa_delinquency
  ,lag(Visa_delinquency, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_Visa_delinquency

  ,lag(Visa_status, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_status
  ,lag(Visa_status, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Visa_status
  ,lag(Visa_status, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_Visa_status

  ,lag(Visa_mfinanciacion_limite_ajustada, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_mfinanciacion_limite
  ,lag(Visa_mfinanciacion_limite_ajustada, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Visa_mfinanciacion_limite
  ,lag(Visa_mfinanciacion_limite_ajustada, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_Visa_mfinanciacion_limite

  ,lag(Visa_Fvencimiento, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_Fvencimiento
  ,lag(Visa_Fvencimiento, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Visa_Fvencimiento
  ,lag(Visa_Fvencimiento, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_Visa_Fvencimiento

  ,lag(Visa_Finiciomora, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_Finiciomora
  ,lag(Visa_Finiciomora, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Visa_Finiciomora
  ,lag(Visa_Finiciomora, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_Visa_Finiciomora

  ,lag(Visa_msaldototal_ajustada, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_msaldototal
  ,lag(Visa_msaldototal_ajustada, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Visa_msaldototal
  ,lag(Visa_msaldototal_ajustada, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_Visa_msaldototal

  ,lag(Visa_msaldopesos_ajustada, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_msaldopesos
  ,lag(Visa_msaldopesos_ajustada, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Visa_msaldopesos
  ,lag(Visa_msaldopesos_ajustada, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_Visa_msaldopesos

  ,lag(Visa_msaldodolares_ajustada, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_msaldodolares
  ,lag(Visa_msaldodolares_ajustada, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Visa_msaldodolares
  ,lag(Visa_msaldodolares_ajustada, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_Visa_msaldodolares

  ,lag(Visa_mconsumospesos_ajustada, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_mconsumospesos
  ,lag(Visa_mconsumospesos_ajustada, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Visa_mconsumospesos
  ,lag(Visa_mconsumospesos_ajustada, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_Visa_mconsumospesos

  ,lag(Visa_mconsumosdolares_ajustada, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_mconsumosdolares
  ,lag(Visa_mconsumosdolares_ajustada, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Visa_mconsumosdolares
  ,lag(Visa_mconsumosdolares_ajustada, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_Visa_mconsumosdolares

  ,lag(Visa_mlimitecompra_ajustada, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_mlimitecompra
  ,lag(Visa_mlimitecompra_ajustada, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Visa_mlimitecompra
  ,lag(Visa_mlimitecompra_ajustada, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_Visa_mlimitecompra

  ,lag(Visa_madelantopesos_ajustada, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_madelantopesos
  ,lag(Visa_madelantopesos_ajustada, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Visa_madelantopesos
  ,lag(Visa_madelantopesos_ajustada, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_Visa_madelantopesos

  ,lag(Visa_madelantodolares_ajustada, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_madelantodolares
  ,lag(Visa_madelantodolares_ajustada, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Visa_madelantodolares
  ,lag(Visa_madelantodolares_ajustada, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_Visa_madelantodolares

  ,lag(Visa_fultimo_cierre, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_fultimo_cierre
  ,lag(Visa_fultimo_cierre, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Visa_fultimo_cierre
  ,lag(Visa_fultimo_cierre, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_Visa_fultimo_cierre

  ,lag(Visa_mpagado_ajustada, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_mpagado
  ,lag(Visa_mpagado_ajustada, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Visa_mpagado
  ,lag(Visa_mpagado_ajustada, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_Visa_mpagado

  ,lag(Visa_mpagospesos_ajustada, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_mpagospesos
  ,lag(Visa_mpagospesos_ajustada, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Visa_mpagospesos
  ,lag(Visa_mpagospesos_ajustada, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_Visa_mpagospesos

  ,lag(Visa_mpagosdolares_ajustada, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_mpagosdolares
  ,lag(Visa_mpagosdolares_ajustada, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Visa_mpagosdolares
  ,lag(Visa_mpagosdolares_ajustada, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_Visa_mpagosdolares

  ,lag(Visa_fechaalta, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_fechaalta
  ,lag(Visa_fechaalta, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Visa_fechaalta
  ,lag(Visa_fechaalta, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_Visa_fechaalta

  ,lag(Visa_mconsumototal_ajustada, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_mconsumototal
  ,lag(Visa_mconsumototal_ajustada, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Visa_mconsumototal
  ,lag(Visa_mconsumototal_ajustada, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_Visa_mconsumototal

  ,lag(Visa_cconsumos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_cconsumos
  ,lag(Visa_cconsumos, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Visa_cconsumos
  ,lag(Visa_cconsumos, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_Visa_cconsumos

  ,lag(Visa_cadelantosefectivo, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_cadelantosefectivo
  ,lag(Visa_cadelantosefectivo, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Visa_cadelantosefectivo
  ,lag(Visa_cadelantosefectivo, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_Visa_cadelantosefectivo

  ,lag(Visa_mpagominimo_ajustado, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_mpagominimo
  ,lag(Visa_mpagominimo_ajustado, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Visa_mpagominimo
  ,lag(Visa_mpagominimo_ajustado, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_Visa_mpagominimo

from competencia_03

## Armado del dataset final

# Guardar el csv con las variables transformadas

In [ ]:
%%sql
copy competencia_03_lags_infla to './datasets/competencia_03_infla_ajustado_lags.csv.gz' (FORMAT CSV, HEADER)